<a href="https://colab.research.google.com/github/ArthurChan-1111/Bitcoin_prediction/blob/main/Master_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!git clone https://github.com/ArthurChan-1111/Bitcoin_prediction.git

%cd Bitcoin_prediction

import pandas as pd

df_bitcoin_by_day = pd.read_csv('btc_1d_data_2018_to_2025.csv', on_bad_lines='skip', lineterminator='\n')


fatal: destination path 'Bitcoin_prediction' already exists and is not an empty directory.
/content/Bitcoin_prediction


In [4]:
df_bitcoin_by_day.describe().T

,count,mean,std,min,25%,50%,75%,max
Open,2638.0,3.007471e+04,2.462686e+04,3.211710e+03,9.141495e+03,2.316573e+04,4.456562e+04,1.061438e+05
High,2638.0,3.076510e+04,2.511586e+04,3.276500e+03,9.304870e+03,2.373610e+04,4.616359e+04,1.083530e+05
Low,2638.0,2.933413e+04,2.411405e+04,3.156260e+03,8.874750e+03,2.270910e+04,4.336132e+04,1.053215e+05
Close,2638.0,3.010409e+04,2.465340e+04,3.211720e+03,9.141368e+03,2.317984e+04,4.466510e+04,1.061438e+05
Volume,2638.0,6.986922e+04,8.017029e+04,4.811771e+02,2.935439e+04,4.436260e+04,7.258744e+04,7.607054e+05
Quote asset volume,2638.0,1.806400e+09,2.007644e+09,1.177017e+07,3.825199e+08,1.126328e+09,2.526423e+09,1.746531e+10
Number of trades,2638.0,1.740156e+06,2.117254e+06,1.241700e+04,4.140775e+05,1.018162e+06,1.931050e+06,1.522359e+07
Taker buy base asset volume,2638.0,3.472933e+04,3.989757e+04,2.313948e+02,1.478038e+04,2.222689e+04,3.584608e+04,3.747756e+05
Taker buy quote asset volume,2638.0,8.943653e+08,9.983054e+08,6.532639e+06,1.907035e+08,5.474179e+08,1.260436e+09,8.783916e+09
Ignore,2638.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
